In [ ]:
!pip install PyPDF2
!pip install spacy
!pip install jsonlines

# spaCy
!python -m spacy download en_core_web_lg

#skils
!pip install git+https://github.com/pandmi/jobzilla_ai/blob/main/jz_skill_patterns.jsonl


In [ ]:
import PyPDF2

text = ''
# Open the PDF file
with open('resume.pdf', 'rb') as file:

    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    # Loop over all pages and extract text

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        page_text = page.extract_text()
        text += page_text

    # Print the extracted text from all pages
    print(text)


In [ ]:
job = []
while True:
    j = input("Enter which type of Job would like to be mostly highligted ?(or 'q' to quit): ")
    if j == 'q':
        break
    if not isinstance(j, str):
        print("Invalid input! Please enter a valid string.")
        continue
    job1 = {"label": "JOB_TYPE", "pattern": j.lower()}
    job2 = {"label": "JOB_TYPE", "pattern": j.upper()}
    job3 = {"label": "JOB_TYPE", "pattern": j.title()}
    job.extend([job1, job2, job3])

print(job)

In [ ]:
main_skill = []
while True:
    msk = input("Enter which type of skill would you like to be mostly highlighted? (or 'q' to quit): ")
    if msk == 'q':
        break
    if not isinstance(msk, str):
        print("Invalid input! Please enter a valid string.")
        continue
    main_skill1 = {"label": "main_skill", "pattern": msk.lower()}
    main_skill2 = {"label": "main_skill", "pattern": msk.upper()}
    main_skill3 = {"label": "main_skill", "pattern": msk.title()}
    main_skill.extend([main_skill1, main_skill2, main_skill3])

print(main_skill)

In [ ]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.matcher import Matcher
import json

nlp = spacy.load("en_core_web_lg")
ruler = nlp.add_pipe("entity_ruler", before="ner")

# Define the patterns first
skills = []
with open("/content/jobzilla_ai/jz_skill_patterns.jsonl") as f:
    for line in f:
        pattern = json.loads(line)
        skills.append(pattern)

# Add the patterns to the ruler
ruler.add_patterns(job)
ruler.add_patterns(main_skill)
ruler.add_patterns(skills)


In [ ]:
matcher = Matcher(nlp.vocab)

edu_patterns = [
    [{"LOWER": {"IN": ["bachelor", "master", "phd"]}}, {"IS_PUNCT": True, "OP": "?"}, {"LOWER": {"IN": ["of", "in"]}}, {"IS_TITLE": True, "OP": "?"}, {"POS": "PROPN"}],
    [{"LOWER": {"IN": ["bs","bca","be","bsc", "ms", "phd"]}}, {"IS_PUNCT": True, "OP": "?"}, {"LOWER": {"IN": ["of", "in"]}}, {"IS_TITLE": True, "OP": "?"}, {"POS": "PROPN"}],
    [{"LOWER": "diploma"}, {"IS_PUNCT": True, "OP": "?"}, {"IS_TITLE": True, "OP": "?"}, {"POS": "PROPN"}],
    [{"LOWER": "certificate"}, {"IS_PUNCT": True, "OP": "?"}, {"IS_TITLE": True, "OP": "?"}, {"POS": "PROPN"}],
    [{"LOWER": {"IN": ["undergraduate", "graduate", "degree"]}}, {"IS_PUNCT": True, "OP": "?"}, {"LOWER": {"IN": ["of", "in"]}}, {"IS_TITLE": True, "OP": "?"}, {"POS": "PROPN"}],
    [{"LOWER": {"IN": ["high", "secondary"]}}, {"LOWER": "school"}, {"IS_PUNCT": True, "OP": "?"}, {"IS_TITLE": True, "OP": "?"}, {"POS": "PROPN"}],
    [{"LOWER": "college"}, {"IS_PUNCT": True, "OP": "?"}, {"IS_TITLE": True, "OP": "?"}, {"POS": "PROPN"}],
    [{"LOWER": "university"}, {"IS_PUNCT": True, "OP": "?"}, {"IS_TITLE": True, "OP": "?"}, {"POS": "PROPN"}],
]
matcher.add("EDUCATION", edu_patterns, on_match=None)

In [ ]:
for pattern in ruler.patterns:
    print(pattern.get("label"))

In [ ]:
from spacy import displacy

doc = nlp(text)

displacy.render(doc, style="ent", jupyter=True)

jobs = []
main_skill = []

for ent in doc.ents:
    if ent.label_ == "JOB_TYPE":
        jobs.append(ent.text.lower())
    elif ent.label_ == "main_skill":
        main_skill.append(ent.text.lower())

jobs = list(set(jobs))
main_skill = list(set(main_skill))

print("Jobs:", ", ".join(jobs))
print("Main Skills:", ", ".join(main_skill))

In [ ]:
import re

for ent in doc.ents:
    if ent.label_ == "PERSON":
        name = ent.text
        break

# Extract the candidate's email address
email = None
email_re = re.compile(r"[^\s@]+@[^\s@]+\.[^\s@]+")
matches = email_re.findall(text)
if matches:
    email = matches[0]

# Extract the candidate's phone number
phone = None
phone_re = re.compile(r"\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}")
matches = phone_re.findall(text)
if matches:
    phone = matches[0]

# Extract the candidate's work experiencs

work_experience = []
for sent in doc.sents:
    if ("work" in sent.text.lower() or "experience" in sent.text.lower() or "internship" in sent.text.lower() or "inter" in sent.text.lower() ) and not ("interests" in sent.text.lower() or "hobbies" in sent.text.lower() or "skill" in sent.text.lower()):
        work_experience.append(sent.text)
work_experience = '\n'.join(work_experience)

SKILL = []
for ent in doc.ents:
    if ent.label_ == "SKILL":
        SKILL.append(ent.text)

jobs = []
main_skill = []

for ent in doc.ents:
    if ent.label_ == "JOB_TYPE":
        jobs.append(ent.text.lower())
    elif ent.label_ == "main_skill":
        main_skill.append(ent.text.lower())

jobs = list(set(jobs))
main_skill = list(set(main_skill))

# Print the extracted information
print(f"Name: {name}")
print("-----------------------------------------")
print(f"Email: {email}")
print("-----------------------------------------")
print(f"Phone: {phone}")
print("----------------------------------------------------------------------------------")
print("Jobs:", ", ".join(jobs))
print("----------------------------------------------------------------------------------")
print("Main Skills:", ", ".join(main_skill))
print("----------------------------------------------------------------------------------")
print("Work Experience:" , work_experience)
print("----------------------------------------------------------------------------------")
matches = matcher(doc)
for match_id, start, end in matches:
    print("Education:" ,doc[start:end])
print("----------------------------------------------------------------------------------")
SKILL = list(set(SKILL))
print("SKILL:")
for edu in SKILL:
    print(f"- {edu}")